In [235]:
from numpy import random, mean, std
from math import sqrt, exp
from matplotlib import pyplot as plt
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter, SUPPRESS

In [275]:
# def initialise(N, n, x_l, x_u):
#     """
#         Initialise population with random values in the range of [x_l, x_u]
#         Initialise the standard deviation in a specific range
#     """
#     all_x = random.uniform(low=x_l, high=x_u, size=(N, n))
#     # ?? what is my eta?
#     all_eta = random.uniform(low=(x_u - x_l) * 0.1,
#                              high=(x_u - x_l) * 0.2, size=(N, n))

#     return list(zip(all_x, all_eta))

def initialise(N, n, x_l, x_u):
    """
        Initialise population with random values in the range of [x_l, x_u]
        Initialise the standard deviation in a specific range
    """
    all_x = random.uniform(low=x_l, high=x_u, size=(N, n))
    eta = std(all_x, axis=0)
    all_eta = [eta] * N

    return list(zip(all_x, all_eta))

In [237]:
def evaluate(pop, f) :
    """
        evaluate the fitness of each individual of the population with the selected function of the three possible functions
    """
    fitnesses = []
    for x, eta in pop:
        fitness = f(x)
        fitnesses.append(fitness)
    return fitnesses

In [238]:
def mutate(pop):
    """
        Mutate the population with the formula for objective variables and strategy parameters seen on slide 14
    """
    n = len(pop[0][0])  # get the dimension of the solution space here.
    N = len(pop)  # the population size
    new_pop = []
    tau = (sqrt(2*sqrt(n)))**-1
    tau_prime = (sqrt(2*n))**-1

    for i in range(N):
        # ?? following 3 lines correct?
        individual = pop[i][0]
        eta = pop[i][1]
        m_individual = individual + eta * random.random()
        m_eta = eta * exp(tau_prime*random.random() + tau*random.random())
        new_pop.append((m_individual, m_eta))

    return new_pop

In [272]:
# def select(pop, fitnesses, N, q=10):
#     """
#         Your docstr here
#     """
#     selected_pop = []
#     for x, eta in pop:
#         selected_pop.append((x, eta))

#     return selected_pop

def select(pop, fitnesses, N, q=10):
    """
        Your docstr here
    """
    selected_pop = []
    fitnesses_ranking = [0] * len(pop)
    for i in range(0, len(pop)):
        fitnessess_indices = random.choice(
            len(fitnesses), size=q)
        for fi in fitnessess_indices:
            if fitnesses[i] < fitnesses[fi]:
                fitnesses_ranking[i] += 1
    top_indices = sorted(range(len(fitnesses_ranking)),
                         key=lambda i: fitnesses_ranking[i], reverse=True)[:N]
    selected_pop = [pop[i] for i in top_indices]

    return selected_pop

In [264]:
def dea(params):
    """
        Your docstr here
    """

    T = params['T']  # get the coresponding parameter from the params dict.
    prev_pop = params["init_pop"]

    t = 0
    mean_fit = [mean(evaluate(prev_pop, params['f']))]
    while t < T:
        # write your code here to implement the DEA. The steps should be 1. mutate, 2. evaluate, 3. select
        #new_pop = mutate(prev_pop)
        #eval_fit = evaluate(new_pop, params['f'])
        #prev_pop = select(new_pop, eval_fit, params['N'])
        m = mean(evaluate(prev_pop, params['f']))
        mean_fit += [m]
        print(f"Mean fitness {t} : ", m)

        t += 1
    return prev_pop, mean_fit

In [265]:
def parseArguments():
    parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter, argument_default=SUPPRESS)
    parser.add_argument("-n", '--length' ,type=int, default=1, help= "The dimension of the solution space")
    parser.add_argument('-T', type=int, default=50, help='Number of generation of the DEA')
    parser.add_argument('-N', type=int, default=100, help='Population size')
    parser.add_argument('-xL', type=float, default=-3.,  help='Lower bound of the the solution space')
    parser.add_argument('-xU', type=float, default=3.,  help='Upper bound of the the solution space')
    parser.add_argument('-q', type=int, default=10,  help='number of mutants for parwise comparison')
    parser.add_argument('--verbose', action="store_true", default=False, help="Print the mean fitness evolution on a standard output " )
    parser.add_argument('-seed', type=int, default=None, help="Seed for the initial population")
    return parser.parse_args()

In [266]:
N = 100 # 100
n = 1 # 1
length = 1 # 1
xL = -2 # -3
xU = 2 # 3
q = 10 # 10

In [267]:
f = lambda x: sum(x**2)
if n==1 and xL == -2 and xU == 2:
    print('Function 1')
    f = lambda x: x**4 + x**3 - x**2 - x
elif n==30 and xL == -100 and xU == 100:
    f = lambda x: sum(x**2.0)
    print('Function 2')
elif n==2 and xL == -30 and xU == 30:
    # f equals the sum from j=1 to j=n-1 of (100(x_j+1-x_j^2)^2 + (x_j-1)^2)
    f = lambda x: sum(100*(x[1:]-x[:-1]**2)**2 + (x[:-1]-1)**2)
    print('Function 3')

Function 1


In [274]:
prev_pop = initialise(N, length, xL, xU)
# prev_pop

mean_fit = [mean(evaluate(prev_pop, f))]

# write your code here to implement the DEA. The steps should be 1. mutate, 2. evaluate, 3. select

new_pop = mutate(prev_pop)
# new_pop

eval_fit = evaluate(new_pop, f)
#eval_fit

prev_pop_1 = select(new_pop, eval_fit, N)
#prev_pop_1

m = mean(evaluate(prev_pop_1, f))
m

3.7744178178301153

In [234]:
type(prev_pop)

list

In [228]:
prev_pop[0]

(array([-0.05142214]), array([0.50767766]))

In [232]:
mean_fit

[2.136352207409729]

In [231]:
new_pop[0]

(array([0.0138951]), array([1.61684066]))

In [230]:
eval_fit[0]

array([0.04864892])

In [229]:
prev_pop_1[0]

array([1.61684066])

In [90]:
def main():
    args = parseArguments()

    if args.length == 1 and args.xL == -2 and args.xU == 2:
        print('Function 1')
        def f(x): return x**4 + x**3 - x**2 - x
        f.__doc__ = 'Function 1 [ x**4 + x**3 - x**2 - x ]'
    elif args.length == 30 and args.xL == -100 and args.xU == 100:
        print('Function 2')
        def f(x): return sum(x**2)
        f.__doc__ = 'Function 2 [ sum(x**2) ]'
    elif args.length == 2 and args.xL == -30 and args.xU == 30:
        print('Function 3')
        def f(x): return sum(100*(x[1:]-x[:-1]**2)**2 + (x[:-1]-1)**2)
        f.__doc__ = 'Function 3 [ sum(100*(x[1:]-x[:-1]**2)**2 + (x[:-1]-1)**2) ]'
    else:
        print('NONE')

    params = {
        'n': args.length,
        'T': args.T,
        'q': args.q,
        'N': args.N,
        'f': f,
        'verbose': args.verbose,
        'xU': args.xU,
        'xL': args.xL
    }

    params['init_pop'] = initialise(args.N, args.length, args.xL, args.xU)

    best_pop, mean_fitnesses = dea(params)
    print("Best population : ", best_pop)
    plt.plot(mean_fitnesses, "o-")
    plt.title("Optimization of the function "+f.__doc__)
    plt.xlabel("Generation(t)")
    plt.ylabel("Population mean fitness")
    plt.show()